In [1]:
from dbconnect.connector import Connection
import pandas as pd

In [239]:
con = Connection('M0neyMa$e')

In [288]:
sql = """

  WITH main_match_key AS(
    SELECT *,CONCAT(squad, ',', match_id) as match_key
    FROM derived.fbref_power_ranking
    WHERE season = 2022
          AND comp IN ('Premier League','Ligue 1','La Liga','Bundesliga','Serie A','Champions League','Europa League')
  )
  SELECT * FROM main_match_key WHERE match_key IN (
  	SELECT CONCAT(squad,',', match_id) as match_key FROM(
      WITH TOPTEN AS (
          SELECT *, ROW_NUMBER() 
          over (
              PARTITION BY squad
              order by date DESC
          ) AS RowNo 
          FROM (
            SELECT DISTINCT(match_id), date, squad 
            FROM derived.fbref_power_ranking 
            WHERE season = 2022
            AND comp IN ('Premier League','Ligue 1','La Liga','Bundesliga','Serie A','Champions League','Europa League')
          ) AS all_data
      )
      SELECT * FROM TOPTEN WHERE RowNo <= 5
    ) AS match_ids
) ORDER BY match_key










"""

team_query = """
SELECT team_name, decorated_name FROM mclachbot_teams
WHERE gender='m'
"""
data=con.query(sql)
team_names = con.query(team_query)

data = pd.merge(data, team_names, how='left', left_on='squad', right_on='team_name')
data['squad']=data['decorated_name']

In [294]:
data.loc[data['player']=='kepa arrizabalaga'].sort_values('date')

,player,rank_position,squad,opponent,date,comp,match_id,season,defending,finishing,keeping,progressing,providing,total,id,minutes,match_key,team_name,decorated_name
2065,kepa arrizabalaga,GK,Chelsea,crystal_palace,2022-10-01,Premier League,372249f3,2022,0.0,0.0,716.666667,0.0,0.0,716.666667,23343,90.0,"chelsea,372249f3",chelsea,Chelsea
2058,kepa arrizabalaga,GK,Chelsea,milan,2022-10-05,Champions League,012a9877,2022,0.0,0.0,1360.000000,0.0,0.0,1360.000000,23345,90.0,"chelsea,012a9877",chelsea,Chelsea
2124,kepa arrizabalaga,GK,Chelsea,wolves,2022-10-08,Premier League,c83cb314,2022,0.0,0.0,1940.000000,0.0,0.0,1940.000000,23348,90.0,"chelsea,c83cb314",chelsea,Chelsea
2102,kepa arrizabalaga,GK,Chelsea,milan,2022-10-11,Champions League,c4c72cb6,2022,0.0,0.0,1230.000000,0.0,0.0,1230.000000,23346,90.0,"chelsea,c4c72cb6",chelsea,Chelsea
2079,kepa arrizabalaga,GK,Chelsea,aston_villa,2022-10-16,Premier League,7113ce7f,2022,0.0,0.0,1990.000000,0.0,0.0,1990.000000,33749,90.0,"chelsea,7113ce7f",chelsea,Chelsea


In [272]:
data.groupby('squad').agg({'comp':lambda x: ','.join(x.unique()).replace('Champions League','').replace('Europa League','').strip(',')})

,comp
squad,
AC Ajaccio,Ligue 1
AC Milan,Serie A
AFC Ajax,
AFC Bournemouth,Premier League
Almeria,La Liga
...,...
Villarreal,La Liga
Werder Bremen,Bundesliga
West Ham United,Premier League


In [282]:
def create_rank(data:pd.DataFrame, metric_to_use:str)->pd.DataFrame:
  domestic_leagues = data.groupby('squad').agg({'comp':lambda x: ','.join(x.unique()).replace('Champions League','').replace('Europa League','').strip(',')}).reset_index().rename(columns={'comp':'domestic league'})
  data = pd.merge(left=data, right=domestic_leagues, on='squad', how='left')
  data = data.loc[data['domestic league']!='']
  aggregated=data.groupby(['player','domestic league','squad']).agg({'defending':'sum','finishing':'sum','progressing':'sum','providing':'sum','keeping':'sum','total':'sum','rank_position':'first'}).reset_index()
  for c in ['finishing','defending','progressing','providing','keeping','total']:
    aggregated[f"{c}_r"]= aggregated[c].rank(pct=True)*100
  aggregated['is_keeper']=aggregated['rank_position']=='GK'
  aggregated=aggregated.drop(labels=['rank_position'],axis=1)
  aggregated['Percentile Rank (Overall)']=aggregated[metric_to_use].rank(pct=True)*100
  for i, g in aggregated.groupby('squad'):
    aggregated.loc[aggregated['squad']==i, 'Rank (Team)'] = g[metric_to_use].rank(ascending=False)
    aggregated.loc[aggregated['squad']==i, 'PCT Rank (Team)'] = g[metric_to_use].rank(pct=True)*100
  return aggregated
  

In [290]:
result = create_rank(data, 'total')
chels = result.loc[result['squad']=='Chelsea'].sort_values('total', ascending=False)
keepers = result.loc[result['is_keeper']].sort_values('total', ascending=False)

In [291]:
chels

,player,domestic league,squad,defending,finishing,progressing,providing,keeping,total,finishing_r,defending_r,progressing_r,providing_r,keeping_r,total_r,is_keeper,Percentile Rank (Overall),Rank (Team),PCT Rank (Team)
1280,mason mount,Premier League,Chelsea,1478.0,1575.0,5299.00,3300.0,0.000000,11652.000000,97.627286,63.766683,93.944637,99.752842,47.478992,97.726149,False,97.726149,1.0,100.000000
1703,ruben loftus cheek,Premier League,Chelsea,2690.0,0.0,5463.00,980.0,0.000000,9133.000000,27.187346,86.159170,94.463668,81.265447,47.478992,93.672763,False,93.672763,2.0,95.238095
1282,mateo kovacic,Premier League,Chelsea,1415.0,0.0,5711.00,990.0,0.000000,8116.000000,27.187346,62.234305,95.254572,81.636184,47.478992,90.805734,False,90.805734,3.0,90.476190
935,jorginho,Premier League,Chelsea,1890.0,800.0,4153.00,550.0,0.000000,7393.000000,88.358873,73.183391,89.471083,63.568957,47.478992,88.235294,False,88.235294,4.0,85.714286
357,conor gallagher,Premier League,Chelsea,1991.0,750.0,3550.00,1085.0,0.000000,7376.000000,87.592684,75.580821,85.912012,84.033613,47.478992,88.185863,False,88.185863,5.0,80.952381
1014,kepa arrizabalaga,Premier League,Chelsea,0.0,0.0,0.00,0.0,7236.666667,7236.666667,27.187346,5.585764,5.907069,9.886307,99.950568,87.296095,True,87.296095,6.0,76.190476
1859,thiago silva,Premier League,Chelsea,3790.0,220.0,1026.50,770.0,0.000000,5806.500000,72.071181,95.007415,60.627781,74.295601,47.478992,78.546713,False,78.546713,7.0,71.428571
993,kalidou koulibaly,Premier League,Chelsea,3630.0,0.0,992.50,240.0,0.000000,4862.500000,27.187346,94.364805,59.317845,41.720217,47.478992,69.995057,False,69.995057,8.0,66.666667
991,kai havertz,Premier League,Chelsea,744.0,950.0,2210.00,605.0,0.000000,4509.000000,92.115670,39.644093,77.508651,66.485418,47.478992,65.694513,False,65.694513,9.0,61.904762
1911,trevoh chalobah,Premier League,Chelsea,3355.0,110.0,490.00,360.0,0.000000,4315.000000,63.000494,92.066238,38.655462,50.790905,47.478992,63.346515,False,63.346515,10.0,57.142857


In [255]:
import numpy as np
KEEPING_RANGE=(-1500,3300)
DEFENDING_RANGE = (-500, 2500)
FINISHING_RANGE=(0,1700)
PROGRESSING_RANGE=(-500, 6000)
PROVIDING_RANGE=(0, 2000)
TOTAL_RANGE = (-1500, 7200)

def rank_formatter(v):
  if v < 10:
    return 'color:grey;font-weight: bold'
  if v < 25:
    return 'color:white;font-weight: bold'
  if v < 50: 
    return 'color:green;font-weight: bold'
  if v < 75: 
    return 'color:blue;font-weight: bold'
  if v < 95:
    return 'color:purple;font-weight: bold'
  if v < 99:
    return 'color:orange;font-weight: bold'
  return 'color:pink;font-weight: bold'

def rank_formatter_based_on_column(df):
  
  mask_10 = df['PCT Rank (Team)']<10
  mask_25 = df['PCT Rank (Team)']<25
  mask_50 = df['PCT Rank (Team)']<50
  mask_75 = df['PCT Rank (Team)']<75
  mask_95 = df['PCT Rank (Team)']<95
  mask_99 = df['PCT Rank (Team)']<99
  mask_100 = df['PCT Rank (Team)']>=99

  df1 = pd.DataFrame('', index=df.index, columns=df.columns)
  
  df1.loc[mask_100, 'Rank (Team)']='color:pink;font-weight: bold'
  df1.loc[mask_99, 'Rank (Team)']='color:orange;font-weight: bold'
  df1.loc[mask_95, 'Rank (Team)']='color:purple;font-weight: bold'
  df1.loc[mask_75, 'Rank (Team)']='color:blue;font-weight: bold'
  df1.loc[mask_50, 'Rank (Team)']='color:green;font-weight: bold'
  df1.loc[mask_25, 'Rank (Team)']='color:white;font-weight: bold'
  df1.loc[mask_10, 'Rank (Team)']='color:grey;font-weight: bold'
  
  return df1

def remove_irrelevant_columns(df):
  
  

  df1 = pd.DataFrame('', index=df.index, columns=df.columns)
  
  df1.loc[df['is_keeper']==False, 'keeping']='color:#000011;background-color:#000011'
  df1.loc[df['is_keeper']==True, ['providing','progressing','finishing','defending']]='color:#000011;background-color:#000011'
  
  return df1


def format(styler:pd.io.formats.style.Styler, num_games):
  styler.set_caption('Top 5 League Power Rankings - Based on last 5 games')
  styler.set_properties(**{'background-color':'#000011','color':'ivory','border': '1px black solid !important'})
  styler.format(precision=0)
  styler.format(lambda x: x.title(), subset=['player'])
  styler.apply(rank_formatter_based_on_column,axis=None)
  
  styler.applymap(lambda x: rank_formatter(x), subset=['Percentile Rank (Overall)'])
  
  
  styler.hide()
  styler.hide(['PCT Rank (Team)','is_keeper']+['finishing_r','defending_r','progressing_r','providing_r','keeping_r','total_r'],axis=1)
  styler.background_gradient(vmin=-TOTAL_RANGE[0]*np.sqrt(num_games),vmax=TOTAL_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['total'])
  styler.background_gradient(vmin=-PROVIDING_RANGE[0]*np.sqrt(num_games),vmax=PROVIDING_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['providing'])
  styler.background_gradient(vmin=-PROGRESSING_RANGE[0]*np.sqrt(num_games),vmax=PROGRESSING_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['progressing'])
  styler.background_gradient(vmin=-FINISHING_RANGE[0]*np.sqrt(num_games),vmax=FINISHING_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['finishing'])
  styler.background_gradient(vmin=-DEFENDING_RANGE[0]*np.sqrt(num_games),vmax=DEFENDING_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['defending'])
  styler.background_gradient(vmin=-KEEPING_RANGE[0]*np.sqrt(num_games),vmax=KEEPING_RANGE[1]*np.sqrt(num_games), cmap='RdYlGn', subset=['keeping'])
  styler.format_index(lambda x: x.title(), axis=1)
  styler.apply(remove_irrelevant_columns,axis=None)
  headers = {
    'selector': 'th',
    'props': [
      ('background-color', '#000022'),
      ('color','ivory'),
      ('font-weight', 'bold'),
      ('border-bottom-style','solid'),
      ('border-bottom-width', '1px'),
      ('border-bottom-color', 'ivory')
    ]
  }
  
  styler.set_table_styles([headers], overwrite=False)
  return styler

In [292]:
format(keepers.sort_values(['total'], ascending=False).head(25).style,5)

Player,Domestic League,Squad,Defending,Finishing,Progressing,Providing,Keeping,Total,Percentile Rank (Overall),Rank (Team)
Ivan Provedel,Serie A,Lazio,0,0,0,0,7869,7869,90,4
Kepa Arrizabalaga,Premier League,Chelsea,0,0,0,0,7237,7237,87,6
Brice Samba,Ligue 1,Lens,0,0,0,0,6768,6768,85,7
Frederik Ronnow,Bundesliga,Union Berlin,0,0,0,0,6096,6096,81,2
Jeremias Ledesma,La Liga,Cadiz,0,0,0,0,5986,5986,80,2
Rafal Gikiewicz,Bundesliga,Augsburg,0,0,0,0,5664,5664,77,3
Pau Lopez,Ligue 1,Marseille,0,0,0,0,5391,5391,75,8
Mark Flekken,Bundesliga,Freiburg,0,0,0,0,5391,5391,75,8
Michele Di Gregorio,Serie A,Monza,0,0,0,0,5338,5338,75,6
Vicente Guaita,Premier League,Crystal Palace,0,0,0,0,5326,5326,74,9


In [249]:
import dataframe_image as dfi
dfi.export(format(chels.sort_values(['total'], ascending=False).style,5), 'table.png')